# LINEAR FEEDBACK SHIFT REGISTERS

## NOTEBOOK 3 - BITSTREAM INVERSION AND VALIDATION

Under certain conditions, given a bitstream generated by an LFSR, this can be inverted to recover the tap positions of the LFSR generating the bitstream. This process is called LFSR inversion. 

The methods in this notebook will successfully invert a bitstream to recover the LFSR **if and only if** the LFSR which generated the stream is maximal. 

#### LIBRARY IMPORTS


In [67]:
from lfsr_library import LFSR, Analyser, Validator

### SAMPLE BITSTREAM

In the following code cell we initialise an LFSR, generate a bitstream and ensure the LFSR is maximal. The bitstream is saved in the variable `bitstream`.

In [48]:
DEGREE: int = 8
TAPS: list[int] = [0, 4, 5, 6]

SEED: int = 0b01100111
ITERATIONS: int = 2**DEGREE

lfsr: LFSR = LFSR(degree=DEGREE, tap_positions=TAPS)
print(f"Feedback polynomial: {lfsr.feedback_polynomial_sp}")

lfsr.generate(bitseq=SEED, iterations=ITERATIONS)
period = lfsr.period
print(f"LFSR period: {period}")

bitstream = lfsr.stream
print(f"Bitstream: {stream}")

Feedback polynomial: x**8 + x**6 + x**5 + x**4 + 1
LFSR period: 255
Bitstream: 11100110111011100101010010100010010110100011001110011110001101100001000101110101111011011111000011010011010110110101000001001110110010010011000000111010010001110001000000010110001111010000111111110010000101001111101010101110000011000101011001100101111110111


### BITSTREAM INVERSION

See the Wikipedia article for the general idea behind inverting bitstreams to recover tap positions for LFSRs. For our purposes, the inverter is a callable method on the `Analyser` class. It passes arguments:

    - a bit stream, type str
    - a degree, type int

From a bitstream and degree, we can solve for the taps as follows. Firstly initialise an `Analyser` with the stream to investigate as follows (recall, this was saved at the variable `bitstream`). 

**Note.** Passing the degree is optional on initialisation.

In [53]:
analyser: Analyser = Analyser(stream=bitstream)

On `analyser` we can call the method `.lin_solve()` to solve for taps. If the degree was not passed at the initialisation stage above, it will need to be passed here.

The bit vector solution is retrievable from `analyser` by calling `.solution`. 
To see the solution in terms of the tap positions,. call `.tap_positions` .

With the stream saved at the variable `bitstream` above and the degree `DEGREE`, we have:

In [57]:
analyser.lin_solve(degree=DEGREE)

print(f"{analyser.solution = }")
print(f"{analyser.tap_positions = }")
print(f"{lfsr.tap_positions = }")

analyser.solution = [1, 0, 0, 0, 1, 1, 1, 0]
analyser.tap_positions = [0, 4, 5, 6]
lfsr.tap_positions = [0, 4, 5, 6]


Evidenlty, see that the tap positions found above by `analyser` coincide with those initialised in `lfsr`.

#### Sub-maximal case

Our method of finding tap positions will not work for sub-maximal LFSRs. Indeed, for LFSRs, a key matrix will be singular over the finite field $\mathbb Z_2$ resulting in an error. This is caught by `.lin_solve()` and displayed to the user `ValueError`. To demonstrate, we will attempt to invert `bitstream` in degree `6` below.

In [61]:
analyser.lin_solve(degree=6)

ValueError: Cannot solve for given bitstream and degree

### ITERATIVE BITSTREAM INVERTER

If we just have a long bitstream and do not know the degree, the `iter_solve()` method will check every possible degree to check for a possible solution. Possible degrees are any degree from `3` to half the length of the bitstream. 

The results of `iter_solve()` are retrieved by passing `.lfsr_solutions`, demonstrated below for `bitstream` from earlier (truncated to its first `20` characters).

In [66]:
analyser: Analyser = Analyser(stream=bitstream[:20])
analyser.iter_solve()
for k, v in analyser.lfsr_solutions.items():
    print(f"Solution for degree {k}: {v}")

Solution for degree 3: {'taps': None, 'tap_positions': None}
Solution for degree 4: {'taps': [0, 1, 1, 1], 'tap_positions': [1, 2, 3]}
Solution for degree 5: {'taps': [1, 1, 1, 0, 1], 'tap_positions': [0, 1, 2, 4]}
Solution for degree 6: {'taps': None, 'tap_positions': None}
Solution for degree 7: {'taps': [0, 1, 0, 0, 0, 0, 1], 'tap_positions': [1, 6]}
Solution for degree 8: {'taps': [1, 0, 0, 0, 1, 1, 1, 0], 'tap_positions': [0, 4, 5, 6]}
Solution for degree 9: {'taps': None, 'tap_positions': None}
Solution for degree 10: {'taps': None, 'tap_positions': None}


Curiously, while we know that the LFSR which generated `bitstream` has degree `8`, solutions in degrees `4, 5` and `7` have also been found.

How accurate are these other solutions? The `Validator` class can be used to investigate further.

### VALIDATION

In the iterative inverter, the bitstream was truncated to its first `20` characters. This allowed for checking if solutions exist from degree `3` to `10`. 

With `Validator`, solutions can then be passed along with the full bitstream. It serves to check how accurately the LFSR corresponding to the tap positions in each solution will regenerate the bitstream. This is implemented through the `.validate()` method.

The `Validator` class requires the following arguments to initialise:

    - the stream, type str
    - the tap positions, type list[int]
    - the degree, type int

We demonstrate below for `bitstream` from earlier.

In [72]:
lfsr_solutions = analyser.lfsr_solutions
for DEGREE, rsults in lfsr_solutions.items():
    if rsults['tap_positions'] != None:
        TAPS = rsults['tap_positions']
        validator = Validator(stream=bitstream, degree=DEGREE, tap_positions=TAPS)
        validator.validate()
        print(f"The Hamming length between the original bitstream and the degree {DEGREE} LFSR with taps at {TAPS} is: {validator.hamming_length}")
        print(f"The Accuracy of the degree {DEGREE} LFSR with taps at {TAPS} is: {100*validator.accuracy:.2f} %\n")

The Hamming length between the original bitstream and the degree 4 LFSR with taps at [1, 2, 3] is: 117
The Accuracy of the degree 4 LFSR with taps at [1, 2, 3] is: 54.47 %

The Hamming length between the original bitstream and the degree 5 LFSR with taps at [0, 1, 2, 4] is: 126
The Accuracy of the degree 5 LFSR with taps at [0, 1, 2, 4] is: 50.97 %

The Hamming length between the original bitstream and the degree 7 LFSR with taps at [1, 6] is: 131
The Accuracy of the degree 7 LFSR with taps at [1, 6] is: 49.03 %

The Hamming length between the original bitstream and the degree 8 LFSR with taps at [0, 4, 5, 6] is: 0
The Accuracy of the degree 8 LFSR with taps at [0, 4, 5, 6] is: 100.00 %



#### CONCLUDING REMARKS

Were we not *a priori* aware that `bitstream` was generated by a degree `8` LFSR, inspection of the above results clearly reveal that the degree `8` LFSR with prescribed taps is the most likely LFSR responsible for generating `bitstream`.